In [1]:
import pandas as pd
import numpy as np
import bqplot as bqp
import ipywidgets as ipw
import datetime
from IPython.display import HTML

In [2]:
# Quick styling fix for bqplot hover style
HTML("""
<style>
    body {
        color: white !important;
    }
</style>
""")

In [3]:
# Load data into a DataFrame
fx_data = pd.read_csv('fx_price_data.csv').set_index('Dates')
fx_data.index = pd.to_datetime(fx_data.index)

# Calculate returns
fx_returns = (((fx_data / fx_data.shift(-1)) - 1)*100)

In [4]:
# Currency pairs for drop-down list
currency_list = [
    'USDSEK BGN Curncy',
    'USDCHF BGN Curncy',
    'USDDKK BGN Curncy',
    'EURUSD BGN Curncy',
    'AUDUSD BGN Curncy',
    'USDJPY BGN Curncy',
    'USDTWD REGN Curncy',
    'NZDUSD BGN Curncy',
    'USDSGD BGN Curncy',
    'USDCAD BGN Curncy',
    'USDKRW REGN Curncy',
    'GBPUSD BGN Curncy',
    'USDNOK BGN Curncy',
    'USDMXN BGN Curncy',
    'USDZAR BGN Curncy',
    'USDBRL REGN Curncy'
]

In [5]:
# Line chart
line_sc_x = bqp.DateScale()
line_sc_y = bqp.LinearScale()
line_mark = bqp.Lines(x=[], y=[],
             scales={'x': line_sc_x, 'y': line_sc_y})
line_fast = bqp.interacts.FastIntervalSelector(scale=line_sc_x, marks=[line_mark])
line_ax_x = bqp.Axis(scale=line_sc_x, label='Date')
line_ax_y = bqp.Axis(scale=line_sc_y, orientation='vertical', label='Price')
line_fig = bqp.Figure(marks=[line_mark], axes=[line_ax_x, line_ax_y], title='Select a currency pair', 
                      interaction=line_fast, layout={'width':'50%', 'display': 'flex', 'flex':'1 1 auto'})

In [6]:
# Scatter chart
scatter_sc_x = bqp.DateScale()
scatter_sc_y = bqp.LinearScale()
scatter_sc_col = bqp.ColorScale()

scatter_tooltip = bqp.Tooltip(fields=['x', 'y'], formats=['%m-%d-%Y', '.2f'])
scatter_scatt = bqp.Scatter(x=[], y=[], color=[], scales={'x': scatter_sc_x, 'y': scatter_sc_y, 'color': scatter_sc_col}, 
                            tooltip=scatter_tooltip, unhovered_style={'opacity': 0.3})
scatter_ax_x = bqp.Axis(scale=scatter_sc_x, label='Date')
scatter_ax_y = bqp.Axis(scale=scatter_sc_y, orientation='vertical', 
                        tick_format='0.2f', label='Return')

scatter_ax_col = bqp.ColorAxis(scale=scatter_sc_col, tick_format='0.2%', label='Returns', orientation='vertical', side='right')
scatter_fig = bqp.Figure(marks=[scatter_scatt], axes=[scatter_ax_x, scatter_ax_y, scatter_ax_col], fig_margin=dict(top=50, bottom=70, left=50, right=100),
                         layout={'width':'50%', 'display': 'flex', 'flex':'1 1 auto'})

In [7]:
# Start and end date values
min_start_date = fx_data.index.min()
max_end_date = fx_data.index.max()

In [8]:
# Defining callback functions
def update_line_chart(event):
    data_to_use = fx_data[[dropdown.value]].loc[(fx_data.index 
                                                 >= pd.to_datetime(start_date.value)) & (fx_data.index 
                                                 <= pd.to_datetime(end_date.value))]
    
    x_data = data_to_use.index
    y_data = data_to_use.values
    line_mark.x = x_data
    line_mark.y = y_data
    line_fig.title = dropdown.value
    
def update_scatter(event):
    start_period, end_period = line_fast.selected
    data_to_use = fx_returns.loc[(fx_returns.index > start_period) & (fx_returns.index <= end_period), dropdown.value]
    
    x_data = data_to_use.index
    y_data = data_to_use.values
    scatter_scatt.x = x_data
    scatter_scatt.y = y_data
    scatter_scatt.color = y_data
    scatter_fig.title = 'Return for period: {} - {}'.format(np.datetime_as_string(start_period)[:10], 
                                                            np.datetime_as_string(end_period)[:10])
    
def calculate_ranking(event):
    start_period, end_period = line_fast.selected
    data_to_use = fx_returns.loc[(fx_returns.index > start_period) & (fx_returns.index <= end_period), :]
    rank = data_to_use.describe().loc[['mean']].T
    rank_dataframe = rank.sort_values(by='mean', ascending=False).rename({'mean':'Top 5 average return for period'},
                                                                         axis=1).head(5).style.applymap(red_green_style)
    rank_dataframe = rank_dataframe.set_table_styles([{'selector': 'tr:hover', 'props': [('background-color', 'grey')], 
                                                       'selector':'td', 'props': [('text-align', 'right')]}]).render()
    ranking_table.value = rank_dataframe
    
def update_scatter_and_rankings(event):
    update_scatter(event)
    calculate_ranking(event)
    
def red_green_style(val):
    color = 'red' if val < 0 else 'lawngreen'
    return 'color: %s' % color

In [9]:
# Graphical user interface with widgets
dropdown = ipw.Dropdown(options=currency_list)
start_date = ipw.DatePicker(value=min_start_date)
end_date = ipw.DatePicker(value=max_end_date)
ranking_table = ipw.HTML()

control_row = ipw.HBox([dropdown,start_date,end_date], layout={'width':'100%', 'display': 'flex', 'flex':'1 1 auto'})
charts_and_table = ipw.HBox([line_fig, scatter_fig], layout={'width':'100%', 'display': 'flex', 'flex':'1 1 auto'})

# Event listeners
dropdown.observe(update_line_chart, 'value')
start_date.observe(update_line_chart, 'value')
end_date.observe(update_line_chart, 'value')
line_fast.observe(update_scatter_and_rankings, 'selected')

update_line_chart(dropdown.value)

ipw.VBox([control_row, charts_and_table, ranking_table], layout={'width':'100%', 'display': 'flex', 'flex':'1 1 auto'})